In [44]:
import numpy as np
import json, os
from nltk.corpus import wordnet as wn
from math import log
import glob
import tqdm
import json

In [45]:
def _getTextFile(langual):
    file_list = glob.glob(f'../data/stopwords/stopwords/*_{langual}.txt')
    files = ",".join(file_list)
    return files
def countMinMaxAver(lines):
    min_len = 10000
    aver_len = 0
    max_len = 0
    for temp in lines:
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)
    aver_len = 1.0 * aver_len / len(lines)
    print('min_len : ' + str(min_len))
    print('max_len : ' + str(max_len))
    print('average_len : ' + str(aver_len))

In [46]:
stop_words=set()
for file in _getTextFile("en").split(","):
    for word in open(file):
        stop_words.add(word.strip())

In [47]:
tagme_json = np.load("../data/wordnet_sememe_entity_sence.npy", allow_pickle=True).tolist()
title_content_lemmatization = np.load("../data/wordnet_lemmatization_sence.npy", allow_pickle=True).tolist()

In [48]:
title_list = list(title_content_lemmatization.keys())

In [49]:
stop_words.add("")
stop_words.add("'s")
stop_words.add("States")
stop_words.add("United")

In [50]:
tagme_sememe_dict_l = {}
for items in tqdm.tqdm(tagme_json):
    if not items:
        continue 
    i, entityList = items.split("\t")
    if entityList == "null" or len(entityList) == 0:
        continue
    entityList = json.loads(entityList)
    entities = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    #entities = [d['title'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    title = title_list[int(i)]
    if title not in tagme_sememe_dict_l.keys():
        tagme_sememe_dict_l[title] = []
    entities = " ".join(entities).strip().split(" ")
    tagme_sememe_dict_l[title] = entities

100%|████████████████████████████████| 227733/227733 [00:03<00:00, 75368.39it/s]


In [51]:
tagme_sememe_dict = {k:v for k,v in tagme_sememe_dict_l.items() if len(v)!=0}

In [52]:
for k, v in tagme_sememe_dict.items():
    def_list = []
    for v_i in v:
        if v_i not in stop_words:
            def_list.append(v_i)
    tagme_sememe_dict[k]=def_list

In [53]:
word_freq = {}
word_set = set()
for doc_words in tagme_sememe_dict.values():
    for word in doc_words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

print("content lexion: ", vocab_size)

content lexion:  29629


In [54]:
word_freq_sorted = sorted(word_freq.items(), key = lambda kv:(kv[1], kv[0]))

In [55]:
sememe_raw = [word for (word, _) in word_freq_sorted[-5000:]]

In [56]:
title_sememe_raw = {}
for k, v in tqdm.tqdm(tagme_sememe_dict.items()):
    sememe = []
    for w in v:
        if w in sememe_raw:
            sememe.append(w)
    if len(sememe) == 0:
        continue
    title_sememe_raw[k] = sememe

print("take max [:5000] in content lexion for sememe (raw)")
countMinMaxAver(title_sememe_raw)

100%|█████████████████████████████████| 227733/227733 [00:40<00:00, 5607.38it/s]

take max [:5000] in content lexion for sememe (raw)
min_len : 5
max_len : 75
average_len : 14.45326050556024


In [57]:
sememe_freq = {}
sememe_set = set()
for doc_sememes in title_sememe_raw.values():
    for sememe in doc_sememes:
        sememe_set.add(sememe)
        if sememe in sememe_freq:
            sememe_freq[sememe] += 1
        else:
            sememe_freq[sememe] = 1

sememe_lexion = list(sememe_set)
sememe_lexion_size = len(sememe_lexion)

sememe_id_map = {}
for i in range(sememe_lexion_size):
    sememe_id_map[sememe_lexion[i]] = i

print("sememe lexion size: ", sememe_lexion_size)

sememe lexion size:  5000


In [58]:
doc_word_freq = {}
for doc_id,(_, doc_words) in enumerate(title_sememe_raw.items()):
    for word in doc_words:
        word_id = sememe_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1
            
word_doc_list = {}
for i,(_, doc_words) in enumerate(title_sememe_raw.items()):
    appeared = set()
    for word in doc_words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)
    
tfidf_word2doc_all = {}
for i,(title, doc_words) in enumerate(title_sememe_raw.items()):
    doc_word_set = set()
    for word in doc_words:
        if word in doc_word_set or word == title:
            continue
        j = sememe_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        idf = log(1.0 * len(title_sememe_raw) / word_doc_freq[sememe_lexion[j]])
        tfidf_word2doc_all[key] = freq * idf
        doc_word_set.add(word)

In [23]:
title_list = [word for word in title_sememe_raw.keys()]
tfidf_filter = {}
for title, tfidf in tfidf_word2doc_all.items():
    if tfidf > 0:
        w, d = title.split(",")
        if title_list[int(w)] not in tfidf_filter.keys():
            tfidf_filter[title_list[int(w)]] = []
        tfidf_filter[title_list[int(w)]].append(sememe_lexion[int(d)])
print("all lexion size:", len(tfidf_filter))

all lexion size: 173194


In [59]:
# this part for add pmi weight to building graph
#title_list = [word for word in title_sememe_raw.keys()]
#tfidf_filter = {}
#for title, tfidf in tfidf_word2doc_all.items():
#    if tfidf > 0:
#        w, d = title.split(",")
#        if title_list[int(w)] not in tfidf_filter.keys():
#            tfidf_filter[title_list[int(w)]] = []
#        pmi_weight = sememe_lexion[int(d)]+":"+str(tfidf)
#        tfidf_filter[title_list[int(w)]].append(pmi_weight)
#print("all lexion size:", len(tfidf_filter))

all lexion size: 173194


In [62]:
#tfidf_filter["apple.n.0"]

['fruit:4.549004087244015',
 'red:5.246527642403759',
 'yellow:4.880575687866226',
 'green:5.59913817555742',
 'skin:5.256445079061105',
 'sweet:6.0782313517909',
 'tart:8.398605986348445',
 'flesh:6.705581357806078']

In [60]:
#print("all lexion size:", len(tfidf_filter))

all lexion size: 173194


In [36]:
#np.save("wordnet_sememe_163340", tfidf_filter)

In [63]:
#sememe_network_pmi = {}
#for k,v in tfidf_filter.items():
#    for vi in v:
#        vi,pmi=vi.split(":")
#        if vi in sememe_network[k]:
#            if k not in sememe_network_pmi.keys():
#                sememe_network_pmi[k] = []
#            sememe_network_pmi[k].append(vi+":"+pmi)

In [65]:
#len(sememe_network_pmi)

In [66]:
#np.save("wordnet_sememe_163340_218", sememe_network_pmi)

In [67]:
#len(title_list)

In [64]:
#title_id_map = {w:i for i,w in enumerate(title_list)}
#max_value = 4
#sememe_network = {}
#for word, sememes in tfidf_filter.items():
#    if len(sememes) != 0:
#        if len(sememes) > max_value:
#            sememe_freq_max_value = {}
#            for sememe in sememes:
#                tfidf_word2doc_all_key = str(title_id_map[word]) +","+ str(sememe_id_map[sememe])
#                sememe_freq_max_value[sememe] = tfidf_word2doc_all[tfidf_word2doc_all_key]
#            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
#            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
#            sememe_network[word] = sememe_lower
#        else:
#            sememe_network[word] = sememes
#    else:
#        continue

In [24]:
title_list = list(title_content_lemmatization.keys())

In [25]:
tagme_sememe_dict = {}
for items in tqdm.tqdm(tagme_json):
    if not items:
        continue 
    i, entityList = items.split("\t")
    if entityList == "null" or len(entityList) == 0:
        continue
    entityList = json.loads(entityList)
    #entities = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    entities = [d['title']+"#"+str(d['link_probability']) for d in entityList if 'title' in d and float(d['rho']) > 0.01]
    #print(i)
    title = title_list[int(i)]
    if title not in tagme_sememe_dict.keys():
        tagme_sememe_dict[title] = []
    tagme_sememe_dict[title] = entities

100%|████████████████████████████████| 227733/227733 [00:03<00:00, 73691.08it/s]


In [26]:
title_list = list(title_content_lemmatization.keys())
max_value = 10
sememe_network = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            tagme_sememe_dict_v = tagme_sememe_dict[word]
            sememe_freq_max_value = {}
            for sememe in sememes:
                for v in tagme_sememe_dict_v:
                    #print(v)
                    s, l = v.split("#")
                    if s == sememe:
                        sememe_freq_max_value[sememe] = float(l)
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network[word] = sememe_lower
        else:
            sememe_network[word] = sememes
    else:
        continue

In [24]:
max_value = 4
sememe_network = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            sememe_freq_max_value = {}
            for sememe in sememes:
                sememe_freq_max_value[sememe] = sememe_freq[sememe]
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network[word] = sememe_lower
        else:
            sememe_network[word] = sememes
    else:
        continue

In [27]:
sememe_network = {k:v for k,v in sememe_network.items() if len(v)!=0}

In [28]:
print("all lexion size:", len(sememe_network))

all lexion size: 171580


In [30]:
sum([len(v) for v in sememe_network.values()])/len(sememe_network)

3.3766464622916423

In [31]:
sum([len(v) for v in tfidf_filter.values()])/len(tfidf_filter)

3.5069979329537975

In [33]:
word_freq = {}
word_set = set()
for words in sememe_network.values():
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

print("vocab_size: ", vocab_size)

vocab_size:  5000


In [34]:
sememe_network["apple.n.0"]

['fruit', 'red', 'yellow', 'green', 'skin', 'sweet', 'tart', 'flesh']

In [38]:
sememe_network = {k:set(v) for k, v in sememe_network.items()}

In [39]:
sememe_network_dict = {}
for k, v in sememe_network.items():
    word = k.split(".")[:-2]
    pos = k.split(".")[-2]
    index = k.split(".")[-1]
    if "".join(word) not in sememe_network_dict.keys():
        sememe_network_dict["".join(word)] = []
    sememe_network_dict["".join(word)].append((pos,v))

In [42]:
sememe_network_dict["apple"]

[('n', {'flesh', 'fruit', 'green', 'red', 'skin', 'sweet', 'tart', 'yellow'}),
 ('n',
  {'Eurasian', 'cultivate', 'edible', 'fruit', 'native', 'tree', 'variety'})]

In [43]:
np.save("sememe_network_dict_en_wordnet_5000",sememe_network)
np.save("sememe_network_cdv_en_wordnet_5000",vocab)